In [10]:
# pip install ipfn


In [11]:
# import pandas as pd
# from ipfn import ipfn
# import numpy as np
# #a=pd.read_csv("/content/2D Ipf - Untitled spreadsheet - Sheet1.csv")
# n=[[[476,410],[472,400],[407,483],[413,413]],[[410,400],[465,472],[460,488],[424,491]]]


# xipp=np.array([4200,3446])
# xpjp=np.array([2005,2000,1958,2043])
# xppk=np.array([4250,4500])
# xijp=np.array([[975,981,974,975],[946,952,947,946]])
# xpjk=np.array([[1200,1183],[1000,950],[900,1050],[950,1109]])

# n=np.array(n)
# aggregates=[xipp,xpjp,xppk,xijp,xpjk]
# dimensions=[[0],[1],[2],[0,1],[1,2]]
# IPF=ipfn.ipfn(n,aggregates,dimensions,convergence_rate=1e-6)
# n=IPF.iteration()
# print(n)


In [12]:
import pandas as pd
import numpy as np
import random
import itertools

np.random.seed(37)
random.seed(37)

height = [
    np.random.normal(5.5, 1.0, 100),
    np.random.normal(5.3, 1.0, 200),
    np.random.normal(5.9, 1.0, 300),
    np.random.normal(5.7, 1.0, 200),
    np.random.normal(5.3, 1.0, 400),
    np.random.normal(5.2, 1.0, 500),
    np.random.normal(5.8, 1.0, 300),
    np.random.normal(5.5, 1.0, 200)
]

#height = normal distribution **(HAVE TO BE FETCHED FROM CENSUS)

demographic = [
    ['white', 'minor', 'male'],
    ['white', 'minor', 'female'],
    ['white', 'adult', 'male'],
    ['white', 'adult', 'female'],
    ['other', 'minor', 'male'],
    ['other', 'minor', 'female'],
    ['other', 'adult', 'male'],
    ['other', 'adult', 'female']
]

data = [[{'race': d[0], 'age': d[1], 'gender': d[2], 'height': h} for h in s]
        for d, s in zip(demographic, height)]
data = list(itertools.chain(*data))

df = pd.DataFrame(data)
df.head()

,race,age,gender,height
0,white,minor,male,5.445536
1,white,minor,male,6.174308
2,white,minor,male,5.846647
3,white,minor,male,4.199654
4,white,minor,male,7.018512


In [13]:
height

[array([5.44553639, 6.17430807, 5.84664703, 4.19965383, 7.01851188,
        6.48982371, 5.7776809 , 5.05141065, 6.46196624, 4.67242136,
        6.03465707, 6.72838619, 6.01959233, 5.43664518, 5.46520664,
        5.54556555, 6.94802513, 7.39350553, 5.9030323 , 5.69242609,
        5.19630695, 4.64022861, 5.23265313, 3.43085207, 4.78986876,
        6.1215503 , 6.72148626, 3.78354676, 5.10414952, 6.77377577,
        5.49033188, 5.36195887, 5.94416423, 3.45293256, 5.27393736,
        7.12795356, 4.70692196, 7.25902821, 5.56047569, 5.85761066,
        5.56656733, 5.86252741, 5.19888782, 3.44721731, 7.17781082,
        5.08553659, 6.46049418, 5.84294779, 6.75711156, 4.41939806,
        4.15047619, 5.1280749 , 5.60206317, 5.01819066, 3.76204747,
        7.09394695, 5.19871762, 7.44644019, 5.42080708, 7.01290178,
        4.66570775, 5.09406981, 5.9255445 , 5.44096293, 4.86709292,
        6.56745242, 5.28866056, 5.20230527, 4.44006558, 6.06891002,
        5.63005544, 4.46625671, 4.55407152, 4.04

In [14]:
df.race.value_counts().sort_index() / df.shape[0]

other    0.636364
white    0.363636
Name: race, dtype: float64

In [15]:
df.age.value_counts().sort_index() / df.shape[0]

adult    0.454545
minor    0.545455
Name: age, dtype: float64

In [16]:
df.gender.value_counts().sort_index() / df.shape[0]

female    0.5
male      0.5
Name: gender, dtype: float64

In [17]:
def get_target_marginals(d):
    factors = list(d.keys())
    targets = [sorted([(k2, v2) for k2, v2 in v.items()]) for k, v in d.items()]
    targets = np.array([[v for _, v in item] for item in targets])
    return factors, targets

def get_table(df, targets):
    factors, target_marginals = get_target_marginals(targets)

    cross_tab = pd.crosstab(df[factors[0]], [df[c] for c in factors[1:]])
    shape = tuple([df[c].unique().shape[0] for c in factors])
    table = cross_tab.values.reshape(shape)

    return factors, target_marginals, table

f, u, X = get_table(df, {
    'race': {'white': 5800, 'other': 4200},
    'age': {'minor': 2800, 'adult': 7200},
    'gender': {'male': 4900, 'female': 5100}
})

X = contigency table\
u = target marginals\
f = helper vector

In [28]:
X

array([[[200, 300],
        [500, 400]],

       [[200, 300],
        [200, 100]]], dtype=int64)

In [18]:
def get_coordinates(M):
    return list(itertools.product(*[list(range(n)) for n in M.shape]))

def get_marginals(M, i):
    coordinates = get_coordinates(M)

    key = lambda tup: tup[0]
    counts = [(c[i], M[c]) for c in coordinates]
    counts = sorted(counts, key=key)
    counts = itertools.groupby(counts, key=key)
    counts = {k: sum([v[1] for v in g]) for k, g in counts}

    return counts

def get_all_marginals(M):
    return np.array([[v for _, v in get_marginals(M, i).items()]
                     for i in range(len(M.shape))])

def get_counts(M, i):
    coordinates = get_coordinates(M)

    key = lambda tup: tup[0]
    counts = [(c[i], M[c], c) for c in coordinates]
    counts = sorted(counts, key=key)
    counts = itertools.groupby(counts, key=key)
    counts = {k: [(tup[1], tup[2]) for tup in g] for k, g in counts}

    return counts

def update_values(M, i, u):
    marg = get_marginals(M, i)
    vals = get_counts(M, i)

    d = [[(c, n * u[k] / marg[k]) for n, c in v] for k, v in vals.items()]
    d = itertools.chain(*d)
    d = list(d)

    return d

def ipf_update(M, u):
    for i in range(len(M.shape)):
        values = update_values(M, i, u[i])
        for idx, v in values:
            M[idx] = v

    o = get_all_marginals(M)
    d = get_deltas(o, u)

    return M, d

def get_deltas(o, t):
    return np.array([np.linalg.norm(o[r] - t[r], 2) for r in range(o.shape[0])])

def get_weights(X, max_iters=50, zero_threshold=0.0001, convergence_threshold=3, debug=True):
    M = X.copy()

    d_prev = np.zeros(len(M.shape))
    count_zero = 0

    for _ in range(max_iters):
        M, d_next = ipf_update(M, u)
        d = np.linalg.norm(d_prev - d_next, 2)

        if d < zero_threshold:
            count_zero += 1

        if debug:
            print(','.join([f'{v:.5f}' for v in d_next]), d)
        d_prev = d_next

        if count_zero >= convergence_threshold:
            break

    w = M / M.sum()
    return w

In [19]:
w = get_weights(X)

758.02375,123.06909,3.16228 767.9557278906121
75.74299,7.28011,3.60555 692.0363565714724
6.70820,2.23607,2.23607 69.23235546955618
2.23607,2.23607,2.23607 4.47213595499958
2.23607,2.23607,2.23607 0.0
2.23607,2.23607,2.23607 0.0
2.23607,2.23607,2.23607 0.0


In [20]:
w

array([[[0.113334  , 0.13604081],
        [0.10423127, 0.0663199 ]],

       [[0.21406422, 0.256677  ],
        [0.0783235 , 0.0310093 ]]])

<b> Sampling </b>

In [21]:
import functools

def get_sampling_weights(df, f, w):
    get_filters = lambda df, fields, values: [df[f] == v for f, v in zip(fields, values)]
    get_total = lambda df, fields, values: df[functools.reduce(lambda a, b: a & b, get_filters(df, fields, values))].shape[0]

    return {k: v / get_total(df, f, k) for k, v in zip(list(itertools.product(*[sorted(df[c].unique()) for c in f])), np.ravel(w))}

def get_samples(df, f, w, n=10_000):
    weights = get_sampling_weights(df, f, w)
    s = df.apply(lambda r: weights[tuple([r[c] for c in f])], axis=1)
    return df.sample(n=n, replace=True, weights=s)

sample_df = get_samples(df, f, w)
sample_df

,race,age,gender,height
1461,other,minor,female,5.337720
2034,other,adult,female,6.243500
1715,other,adult,male,5.261089
310,white,adult,male,5.682793
1884,other,adult,male,5.192252
...,...,...,...,...
2140,other,adult,female,6.026459
732,white,adult,female,5.155105
34,white,minor,male,5.273937
117,white,minor,female,5.599598


In [22]:
ct = pd.crosstab(sample_df.race, [sample_df.age, sample_df.gender])
ct #cross-tabulation of the resulting sampled matrix.

age     adult        minor     
gender female  male female male
race                           
other    1165  1344   1041  628
white    2225  2492    791  314

In [23]:
sample_df.race.value_counts().sort_index() / sample_df.shape[0]

other    0.4178
white    0.5822
Name: race, dtype: float64

In [24]:
sample_df.age.value_counts().sort_index() / sample_df.shape[0]

adult    0.7226
minor    0.2774
Name: age, dtype: float64

In [25]:
sample_df.gender.value_counts().sort_index() / sample_df.shape[0]

female    0.5222
male      0.4778
Name: gender, dtype: float64